In [15]:
! pip install langchain-community
! pip install chromadb
! pip install pypdf
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.0 MB/s eta 0:00:00


## Generate Embeddings and Store to DB

In [16]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [17]:
loaders = [PyPDFLoader('./srs_doc.pdf')]

In [18]:
docs    = []
for file in loaders: docs.extend(file.load())

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(docs)
embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device' : 'cpu'})

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
vectorstore = Chroma.from_documents(docs, embedding_function, persist_directory='./chroma_db_nccn')

In [23]:
print(vectorstore._collection.count())

89


## RAG

In [36]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import google.generativeai as genai

In [37]:
GEMINI_API_KEY = ""
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

In [38]:
def generate_rag_prompt(query, context):
  escaped = context.replace("'", "").replace('"', '').replace("\n", " ")
  prompt = (f"""
  You are a helpful and informative bot that answers questions using text from reference context included below. \
  Be sure to respond in a complete sentence, being comprenhensive, including all relevant background information. \
  However, you are talinkg to a non-technical audience, so be sure to break down complicated concepts and \
  strike friendly and conversational tone. \
  If the context is irrelevant to the answer, you may ignore it.

  USER QUESTION: '{query}'
  CONTEXT: '{context}'

  ANSWER:
  """)
  return prompt

In [39]:
def get_relevant_context_from_db(query):
  context = ""
  embedding_function = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device' : 'cpu'})
  vector_db = Chroma(persist_directory='./chroma_db_nccn', embedding_function=embedding_function)
  search_results = vector_db.similarity_search(query, k=6)
  for result in search_results:
    context += result.page_content + "\n"
  return context

In [40]:
def generate_answer(prompt):
  answer = model.generate_content(prompt)
  return answer.text

In [41]:
while True:
  print("="*50)
  print("What would you like to ask?")
  query = input("Query: ")
  context = get_relevant_context_from_db(query)
  prompt = generate_rag_prompt(query, context)
  answer = generate_answer(prompt)
  print(answer)

What would you like to ask?
Query: What does the 6th functional requirement talk about?
The 6th functional requirement states that the system should provide real-time feedback and status updates during administrative actions, such as progress indicators and confirmation messages. This means that users should be kept informed about the progress of their actions and receive confirmation that their actions have been completed successfully. 

What would you like to ask?
Query: What does the Rights Management and Tracking feature provide on the platform?
The Rights Management and Tracking feature on the platform ensures that the licensing of music is legally sound and compliant with copyright laws. This feature involves tracking the usage rights of each song, managing license agreements, and providing clear documentation of the terms under which music can be used. 

What would you like to ask?
Query: how many money is reserved for the contingency buffer? what's the percentage?
The contingen

KeyboardInterrupt: Interrupted by user